In [3]:
import sklearn

import numpy as np
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# Data

In this dataset, the first column is dependent variable $y$, other columns include independent variables. We would like to build appropriate models from predicting $y$.

**Task 1**: 
1. Split the dataset into training (90%) and test set (10%). (1 pt)
2. Use `StandardScaler` function in `sklearn.preprocessing` to standardize X variables. (2 pts)
3. Perform linear regression to fit the training set, then calculate the test MSE. (2 pts)
4. Perform 5-fold cross validation to choose a best model from the following 3 models. You need to obtain cross validation MSE. (5 pts)
   - $y \sim X1+X2$
   - $y \sim X1+X2+X3$
   - $y \sim X1+X2+X3+X4+X5$
5. Calculate the test MSE by using the best model obtained above. You need to refit the model by using all the training set. Is this new MSE less than that in 3? (3 pts)

**Task 2**
1. Perform LASSO regression to select variables. Choose the best parameter $\alpha$ using grid search. (3 pts)
2. With the best parameter $\alpha$, which variables are deleted (the variables with coefficient equal to 0)? (2 pts)
3. Calculate the test MSE by using the LASSO model. (2 pts)

# Task 1

##  1-3

In [4]:
import pandas as pd
df = pd.read_csv("data.csv")
df.head()

,y,X1,X2,X3,X4,X5,X6,X7,X8,X9
0,280.510607,2.940073,8.644027,25.414066,74.719200,219.679873,645.874775,1898.918725,5582.958904,16414.304470
1,214.423378,3.495544,12.218826,42.711441,149.299709,521.883665,1824.267176,6376.805704,22290.403251,77917.079511
2,7.866565,0.940760,0.885029,0.832600,0.783276,0.736875,0.693222,0.652156,0.613522,0.577177
3,21.064747,0.219043,0.047980,0.010510,0.002302,0.000504,0.000110,0.000024,0.000005,0.000001
4,-68.229070,1.025095,1.050819,1.077189,1.104221,1.131931,1.160337,1.189455,1.219304,1.249902


In [5]:
from sklearn.model_selection import train_test_split

X = df.drop('y', axis=1)
y = df['y'].copy()

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.1, random_state=42)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

std = StandardScaler()

X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

y_train = y_train.to_numpy().reshape(-1,1)
y_test = y_test.to_numpy().reshape(-1,1)

lin_reg = LinearRegression()

lin_reg.fit(X_train_std, y_train)

y_train_pred = lin_reg.predict(X_train_std)
train_mse = mean_squared_error(y_train, y_train_pred)

y_test_pred = lin_reg.predict(X_test_std)

test_mse_1 = mean_squared_error(y_test, y_test_pred)

print(f"training mse: {train_mse}")
print(f"test mse: {test_mse_1}")

training mse: 2536.4404320357744
test mse: 3167.606922457756


## 4, 5

In [11]:
def func_mean_val_mse(model, X, y, K=5):
    np.random.seed(2024)
    shuffled_indices = np.random.permutation(len(X))
    average_size = int(np.around(len(X)/K, 0))

    train_mses, val_mses = [], []
    for k in range(K):
        if k<K-1:
            test_indices = shuffled_indices[k*average_size:(k+1)*average_size]
        else:
            test_indices = shuffled_indices[-average_size:]    

        # train_indices are indices that are not in test_indices
        train_indices = shuffled_indices[[shuffled_indices[i] not in test_indices for i in range(len(shuffled_indices))]]

        train_set, val_set = X[train_indices], X[test_indices]

        y_train, y_val = y[train_indices], y[test_indices]

        model.fit(train_set, y_train)
        
        # training mse
        train_pred = model.predict(train_set)
        train_mse = mean_squared_error(y_train, train_pred)
        train_mses.append(train_mse)
        
        # validation mse
        val_pred = model.predict(val_set)
        val_mse = mean_squared_error(y_val, val_pred)
        val_mses.append(val_mse)

    return np.mean(train_mses), np.mean(val_mses)

In [8]:
X_train_std.shape

(180, 9)

In [16]:
# model Y~X1+X2
cols = [0, 1]
X_train_temp = X_train_std[:, cols]

# model
lin_reg = LinearRegression()

mean_train_mse, mean_val_mse = func_mean_val_mse(lin_reg, X_train_temp, y_train)

print(f"mean_train_mse: {mean_train_mse}")
print(f"mean_val_mse: {mean_val_mse}")


mean_train_mse: 4202.7095100493925
mean_val_mse: 4467.823686337683


In [17]:
# model Y~X1+X2+X3
cols = [0, 1, 2]
X_train_temp = X_train_std[:, cols]

# model
lin_reg = LinearRegression()

mean_train_mse, mean_val_mse = func_mean_val_mse(lin_reg, X_train_temp, y_train)

print(f"mean_train_mse: {mean_train_mse}")
print(f"mean_val_mse: {mean_val_mse}")


mean_train_mse: 2617.9940874150643
mean_val_mse: 2795.4552825553264


In [18]:
# model Y~X1+X2+X3+X4+X5
cols = [0, 1, 2, 3, 4]
X_train_temp = X_train_std[:, cols]

# model
lin_reg = LinearRegression()

mean_train_mse, mean_val_mse = func_mean_val_mse(lin_reg, X_train_temp, y_train)

print(f"mean_train_mse: {mean_train_mse}")
print(f"mean_val_mse: {mean_val_mse}")


mean_train_mse: 2529.6049198223614
mean_val_mse: 2733.8919921040706


So best model is Y~X1+X2+X3+X4+X5

In [19]:
# data
cols = [0, 1, 2, 3, 4]
X_train_temp = X_train_std[:, cols]

# model
lin_reg = LinearRegression()
lin_reg.fit(X_train_temp, y_train)

# test mse 
X_test_temp = X_test_std[:,cols]
y_test_pred = lin_reg.predict(X_test_temp)
test_mse = mean_squared_error(y_test, y_test_pred)

print(f"test_mse: {test_mse}")


test_mse: 3144.056435679504


# Task 2

**Task 2**
1. Perform LASSO regression to select variables. Choose the best parameter $\alpha$ using grid search.
2. Calculate the test MSE by using the LASSO model.

In [12]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

params = {'alpha': [0,0.01,0.1,1,10,100, 200, 300, 400]} # 0.1
# params = {'alpha': [0.01,0.05, 0.1,0.5, 1]} # 0.1
# params = {'alpha': [0.05, 0.08, 0.1,0.2, 0.3, 0.4, 0.5]} # 0.2
# params = {'alpha': [0.1, 0.15, 0.2, 0.25, 0.3]} # 0.15
# params = {'alpha': [0.15, 0.16, 0.17, 0.18, 0.19, 0.2]}  # 0.18

gridcv_lasso_model = GridSearchCV(Lasso(),params,cv=5)
gridcv_lasso_model.fit(X_train_std, y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.793e+05, tolerance: 1.258e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0, 0.01, 0.1, 1, 10, 100, 200, 300, 400]})

In [13]:
gridcv_lasso_model.best_params_

{'alpha': 0.1}

In [14]:
gridcv_lasso_model.best_estimator_.coef_

array([  24.50463329,  181.06648081,    0.        ,  -36.43110234,
       -105.97823825,  -50.24111262,   -0.        ,    0.        ,
          1.6198043 ])

In [15]:
lass_pred = gridcv_lasso_model.best_estimator_.predict(X_test_std)

mean_squared_error(y_test, lass_pred)

3217.0527049253315